# Multiple dispatch

In this notebook we'll explore **multiple dispatch**, which is a key feature of Julia.

Multiple dispatch makes software *generic* and *fast*!

#### Starting with the familiar

To understand multiple dispatch in Julia, let's start with what we've already seen.

We can declare functions in Julia without giving Julia any information about the types of the input arguments that function will receive:

In [1]:
f(x) = x^2

f (generic function with 1 method)

and then Julia will determine on its own which input argument types make sense and which do not:

In [2]:
f(10)

100

In [3]:
f([1, 2, 3])

MethodError: MethodError: no method matching ^(::Array{Int64,1}, ::Int64)
Closest candidates are:
  ^(!Matched::Float16, ::Integer) at math.jl:795
  ^(!Matched::Missing, ::Integer) at missing.jl:120
  ^(!Matched::Missing, ::Number) at missing.jl:93
  ...

#### Specifying the types of our input arguments

However, we also have the *option* to tell Julia explicitly what types our input arguments are allowed to have.

For example, let's write a function `foo` that only takes strings as inputs.

In [4]:
foo(x::String, y::String) = println("My inputs x and y are both strings!")

foo (generic function with 1 method)

We see here that in order to restrict the type of `x` and `y` to `String`s, we just follow the input argument name by a double colon and the keyword `String`.

Now we'll see that `foo` works on `String`s and doesn't work on other input argument types.

In [5]:
foo("hello", "hi!")

My inputs x and y are both strings!


In [6]:
foo(3, 4)

MethodError: MethodError: no method matching foo(::Int64, ::Int64)

To get `foo` to work on integer (`Int`) inputs, let's tack `::Int` onto our input arguments when we declare `foo`.

In [7]:
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")

foo (generic function with 2 methods)

In [8]:
foo(3, 4)

My inputs x and y are both integers!


Now `foo` works on integers! But look, `foo` also still works when `x` and `y` are strings!

In [9]:
foo("hello", "hi!")

My inputs x and y are both strings!


This is starting to get to the heart of multiple dispatch. When we declared

```julia
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")
```
we didn't overwrite or replace
```julia
foo(y::String, y::String)```

Instead, we just added an additional ***method*** to the ***generic function*** called `foo`.

A ***generic function*** is the abstract concept associated with a particular operation.

For example, the generic function `+` represents the concept of addition.

A ***method*** is a specific implementation of a generic function for *particular argument types*.

For example, `+` has methods that accept floating point numbers, integers, matrices, etc.

We can use the `methods` to see how many methods there are for `foo`.

In [10]:
methods(foo)

# 2 methods for generic function "foo":
[1] foo(x::Int64, y::Int64) in Main at In[7]:1
[2] foo(x::String, y::String) in Main at In[4]:1

methods(+) returns 163 methods...

So, we now can call `foo` on integers or strings. When you call `foo` on a particular set of arguments, Julia will infer the types of the inputs and dispatch the appropriate method. *This* is multiple dispatch.

Multiple dispatch makes our code generic and fast. Our code can be generic and flexible because we can write code in terms of abstract operations such as addition and multiplication, rather than in terms of specific implementations. At the same time, our code runs quickly because Julia is able to call efficient methods for the relevant types.

To see which method is being dispatched when we call a generic function, we can use the @which macro:

In [21]:
@which foo(3, 4)

foo(x::Int64, y::Int64) in Main at In[7]:1

In [22]:
@which 3.0 + 3.0

+(x::Float64, y::Float64) in Base at float.jl:395

Given that a method written specifically for floating point numbers is dispatched on `3.0 + 3.0`, the LLVM code generated is extremely terse:

In [23]:
@code_llvm 3.0 + 3.0


; Function +
; Location: float.jl:395
define double @"julia_+_34566"(double, double) {
top:
  %2 = fadd double %0, %1
  ret double %2
}


Note that Julia is fast even when we write generic function definitions because, at the end of the day, specific/tailored methods are called under the hood.

For example, note that we can declare the adding function `myadd` without providing any type annotations - 

In [24]:
myadd(x, y) = x + y

myadd (generic function with 1 method)

and though we haven't constrained the types of `x` and `y`, we'll see that the LLVM code generated for `myadd(3.0, 3.0)` looks like that of `3.0 + 3.0`

In [25]:
@code_llvm myadd(3.0, 3.0)


; Function myadd
; Location: In[24]:1
define double @julia_myadd_35761(double, double) {
top:
; Function +; {
; Location: float.jl:395
  %2 = fadd double %0, %1
;}
  ret double %2
}
